In [2]:
import pandas as pd
import numpy as np

from scipy.stats import ks_2samp
#import statsmodels.stats.multitest as multi

import os.path

from helpers import bh

In [3]:
df_control = pd.read_csv('/Users/zivaskof/Documents/MATLAB/mag/fluxSampels/tInitSampels/tINIT_flux_sample_KM_001N.csv')
df_kd = pd.read_csv('/Users/zivaskof/Documents/MATLAB/mag/fluxSampels/tInitSampels/tINIT_flux_sample_KM_001T.csv')

In [4]:
reactions = sorted(list(set(df_control.columns) | set(df_kd.columns)))
len(reactions) # število reakcij

6055

In [5]:
df = pd.DataFrame(columns=['reaction', 'FC', 'p', 'q', 'enrichment', 'changed'])
df['reaction']=reactions

n_samples = df_control.shape[0]

# sprehodimo se čez vse reakcije
for reaction in reactions:
    if reaction in df_control.columns:
        control = df_control[reaction].values
    else:
        # če reakcije ni v kontrolni skupini, ji pripišemo same ničle
        control = np.zeros(n_samples)
        
    if reaction in df_kd.columns:
        kd = df_kd[reaction].values
    else:
        # če reakcije ni v kd skupini, ji pripišemo same ničle
        kd = np.zeros(n_samples)
        
    # iztračunamo sredino za kontrolo in kd
    mean_control = np.mean(control)
    mean_kd = np.mean(kd)
    
    # izračunamo FC - fold change in signifikanco z uporabo 2 sample Kolmogorov-Smirnov testa
    if mean_control != 0 or mean_kd != 0:
        FC = mean_kd-mean_control/(abs(mean_kd + mean_control))
        p = ks_2samp(control,kd)[1]
    else:
        FC = 0
        p = 1     
        
    df.loc[df['reaction']==reaction, 'FC'] = FC
    df.loc[df['reaction']==reaction, 'p'] = p
    
    
# korigiramo p vrednosti za večkratno testiranje - FDR korekcija
df['q'] = bh(df['p'])

# signifikanca zahteva vsaj 10-kratno up-/down-regulacijo
df.loc[(df['FC'] >= 0.82) & (df['q'] < 0.05),'enrichment'] = 1
df.loc[(df['FC'] <= -0.82) & (df['q'] < 0.05),'enrichment'] = -1
df.loc[~df['enrichment'].isna(),'changed'] = 1
df = df.fillna(0)

In [6]:
df[df.enrichment == -1]


,reaction,FC,p,q,enrichment,changed
5,MAR00018,-1.000000,3.580820e-149,6.520862e-149,-1,1
11,MAR00040,-1.220460,0.000000e+00,0.000000e+00,-1,1
12,MAR00047,-1.195181,0.000000e+00,0.000000e+00,-1,1
13,MAR00048,-1.195181,0.000000e+00,0.000000e+00,-1,1
30,MAR00101,-1.000000,2.386293e-165,4.509677e-165,-1,1
...,...,...,...,...,...,...
6014,MAR13012,-0.964294,8.575383e-250,2.062930e-249,-1,1
6051,MAR13081,-0.862338,9.185171e-122,1.598167e-121,-1,1
6052,MAR13082,-5.207502,6.118730e-288,1.639332e-287,-1,1
6053,MAR13083,-1.000000,2.523014e-285,6.691567e-285,-1,1


In [7]:
df[df.enrichment == 1]

,reaction,FC,p,q,enrichment,changed
4,MAR00015,2.315548,1.211020e-203,2.536397e-203,1,1
7,MAR00025,542.784977,4.680860e-265,1.164924e-264,1,1
8,MAR00030,542.784977,4.680860e-265,1.164924e-264,1,1
9,MAR00031,137.742496,6.914010e-26,1.070973e-25,1,1
10,MAR00032,137.742496,6.914010e-26,1.070973e-25,1,1
...,...,...,...,...,...,...
5960,MAR12779,1.000000,6.049929e-147,1.094809e-146,1,1
5962,MAR12781,1.863829,1.445157e-197,2.965240e-197,1,1
5989,MAR12893,1.000000,6.049929e-147,1.094809e-146,1,1
5997,MAR12920,1.863829,1.445157e-197,2.965240e-197,1,1


In [8]:
df[df.enrichment == 0]

,reaction,FC,p,q,enrichment,changed
0,MAR00001,0.0,1.0,1.0,0,0
1,MAR00002,0.0,1.0,1.0,0,0
2,MAR00003,0.0,1.0,1.0,0,0
3,MAR00006,0.0,1.0,1.0,0,0
6,MAR00019,0.0,1.0,1.0,0,0
...,...,...,...,...,...,...
6046,MAR13065,0.0,1.0,1.0,0,0
6047,MAR13066,0.0,1.0,1.0,0,0
6048,MAR13069,0.0,1.0,1.0,0,0
6049,MAR13070,0.0,1.0,1.0,0,0


In [9]:
import pandas as pd
import numpy as np

from scipy.stats import hypergeom
#import statsmodels.stats.multitest as multi
from helpers import bh

import cobra

import os.path

In [12]:
model = cobra.io.read_sbml_model('Recon3D.xml')

In [13]:
model_mat = cobra.io.load_matlab_model('Recon3D.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'Recon3D.mat'

In [14]:
reactions_subsystems = {}
for r in model.reactions:
    reactions_subsystems[r.id] = r.subsystem

In [23]:
df_subsystems = pd.DataFrame()
df_subsystems['subsystem'] = reactions_subsystems.keys()
df_subsystems['reaction'] = reactions_subsystems.keys()
df_subsystems.to_csv('subsystems.csv', index=False)

In [25]:
df_subsystems = pd.read_csv('subsystems.csv')
subsystems = df_subsystems.subsystem.unique()
df_subsystems.head()

,subsystem,reaction
0,24_25DHVITD3tm,24_25DHVITD3tm
1,25HVITD3t,25HVITD3t
2,COAtl,COAtl
3,EX_5adtststerone_e,EX_5adtststerone_e
4,EX_5adtststerones_e,EX_5adtststerones_e


In [26]:
df_reactions = df

In [27]:
df_enrichment = pd.DataFrame(columns=["subsystem", "p_up", "p_down", "q_up", "q_down", "enrichment", "p_changed", "q_changed", "changed"])
df_enrichment["subsystem"] = subsystems

M = len(df_reactions) # number of different reactions in pairs of models
n_up = sum(df_reactions.enrichment == 1) # number of upregulated reactions in models
n_down = sum(df_reactions.enrichment == -1)  # number of downregulated reactions in models
n_changed = sum(df_reactions.changed == 1)  # number of changed reactions in models

for subsystem in subsystems:
    subsystem_reactions = df_subsystems.loc[df_subsystems.subsystem == subsystem,'reaction'].values
    df_sub = df_reactions[df_reactions['reaction'].isin(subsystem_reactions)]
        
    #if not take_all:
    # option 1: take only remaining reactions
    N = len(df_sub) # number of reactions in a subsystem
    #else:
    #    # option 2: take all reactions from the original model
    #    N = len(df_subs[df_subs.subsystem == subsystem])
    k_up = sum(df_sub.enrichment == 1)# number of upregulated reactions in a subsystem
    k_down = sum(df_sub.enrichment == -1)# number of downregulated reactions in a subsystem
    k_changed = sum(df_sub.changed == 1)# number of changed reactions in a subsystem
    
    if n_up:         
        p_up = 1 - hypergeom.cdf(k_up-1, M, n_up, N)                
    else:
        p_up = 1.0
        
    if n_down:         
        p_down = 1 - hypergeom.cdf(k_down-1, M, n_down, N)                
    else:
        p_down = 1.0
        
    if n_changed:
        p_changed = 1 - hypergeom.cdf(k_changed, M, n_changed, N)                
    else:
        p_changed = 1
        
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_up'] = p_up
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_down'] = p_down
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_changed'] = p_changed
    

    
df_enrichment['q_up'] = bh(df_enrichment['p_up'])
df_enrichment['q_down'] = bh(df_enrichment['p_down'])
df_enrichment['q_changed'] = bh(df_enrichment['p_changed'])

    
df_enrichment.loc[(df_enrichment['q_up']<0.05) & (df_enrichment['q_up']<df_enrichment['q_down']),'enrichment'] = 1
df_enrichment.loc[(df_enrichment['q_down']<0.05) & (df_enrichment['q_down']<=df_enrichment['q_up']),'enrichment'] = -1
df_enrichment.loc[(df_enrichment['q_changed']<0.05),'changed'] = 1

df_enrichment = df_enrichment.fillna(0)

In [28]:
df_enrichment

,subsystem,p_up,p_down,q_up,q_down,enrichment,p_changed,q_changed,changed
0,24_25DHVITD3tm,1.0,1.0,1.0,1.0,0,0.0,0.0,1
1,25HVITD3t,1.0,1.0,1.0,1.0,0,0.0,0.0,1
2,COAtl,1.0,1.0,1.0,1.0,0,0.0,0.0,1
3,EX_5adtststerone_e,1.0,1.0,1.0,1.0,0,0.0,0.0,1
4,EX_5adtststerones_e,1.0,1.0,1.0,1.0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
10595,ACMPGLUTTRsc,1.0,1.0,1.0,1.0,0,0.0,0.0,1
10596,FVSCOAhc,1.0,1.0,1.0,1.0,0,0.0,0.0,1
10597,MDZGLChr,1.0,1.0,1.0,1.0,0,0.0,0.0,1
10598,TMACMPhr,1.0,1.0,1.0,1.0,0,0.0,0.0,1
